## Install the package dependencies before running this notebook

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
import matplotlib.pyplot as plt

"""
    number of trajectories in each city
    # austin --  train: 43041 test: 6325 
    # miami -- train: 55029 test:7971
    # pittsburgh -- train: 43544 test: 6361
    # dearborn -- train: 24465 test: 3671
    # washington-dc -- train: 25744 test: 3829
    # palo-alto -- train:  11993 test:1686

    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds
    
"""

'\n    number of trajectories in each city\n    # austin --  train: 43041 test: 6325 \n    # miami -- train: 55029 test:7971\n    # pittsburgh -- train: 43544 test: 6361\n    # dearborn -- train: 24465 test: 3671\n    # washington-dc -- train: 25744 test: 3829\n    # palo-alto -- train:  11993 test:1686\n\n    trajectories sampled at 10HZ rate, input 5 seconds, output 6 seconds\n    \n'

## Create training, validation, and test datasets

In [3]:
#This code is inspired by the code from the Week 7 Discussion
from glob import glob
import pickle
import numpy as np

ROOT_PATH = "./"

cities = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
splits = ["train", "test"]

def get_city_trajectories(city="palo-alto", split="train", normalized=False):
    f_in = ROOT_PATH + "train" + "/" + city + "_inputs"
    f_out = ROOT_PATH + "train" + "/" + city + "_outputs"
    
    inputs = None
    outputs = None
    
    if city=="all":
        allInputs = np.zeros((0,50,2))
        allOutputs = np.zeros((0,60,2))
        for city in cities:
            if split=="train":
                f_in = ROOT_PATH + split + "/" + city + "_inputs"
                inputs = pickle.load(open(f_in, "rb"))
                n = len(inputs)
                allInputs = np.concatenate((allInputs, np.asarray(inputs)[:int(n * 0.8)]))

                f_out = ROOT_PATH + split + "/" + city + "_outputs"
                outputs = pickle.load(open(f_out, "rb"))
                allOutputs = np.concatenate((allOutputs, np.asarray(outputs)[:int(n * 0.8)]))

            elif split == 'val':
                f_in = ROOT_PATH + 'train' + "/" + city + "_inputs"
                inputs = pickle.load(open(f_in, "rb"))
                n = len(inputs)
                allInputs = np.concatenate((allInputs, np.asarray(inputs)[int(n * 0.8):]))

                f_out = ROOT_PATH + 'train' + "/" + city + "_outputs"
                outputs = pickle.load(open(f_out, "rb"))
                allOutputs = np.concatenate((allOutputs, np.asarray(outputs)[int(n * 0.8):]))

            else:
                f_in = ROOT_PATH + split + "/" + city + "_inputs"
                inputs = pickle.load(open(f_in, "rb"))
                n = len(inputs)
                allInputs = np.concatenate((allInputs, np.asarray(inputs)))
                
        if (normalized):
            allInputs = (allInputs - np.min(allInputs))/(np.max(allInputs) - np.min(allInputs))
            allOutputs = (allOutputs - np.min(allOutputs))/(np.max(allOutputs) - np.min(allOutputs))
        return allInputs, allOutputs
    
    if split=="train":
        inputs = pickle.load(open(f_in, "rb"))
        n = len(inputs)
        inputs = np.asarray(inputs)[:int(n * 0.8)]
        
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)[:int(n * 0.8)]
    
    elif split=="val":
        inputs = pickle.load(open(f_in, "rb"))
        n = len(inputs)
        inputs = np.asarray(inputs)[int(n * 0.8):]
        
        outputs = pickle.load(open(f_out, "rb"))
        outputs = np.asarray(outputs)[int(n * 0.8):]
    
    else:
        f_in = ROOT_PATH + spiit + "/" + city + "_inputs"
        f_out = ROOT_PATH + split + "/" + city + "_outputs"
        inputs = pickle.load(open(f_in, "rb"))
        n = len(inputs)
        inputs = np.asarray(inputs)
        return inputs

    return inputs, outputs

class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, city: str, split:str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.transform = transform

        self.inputs, self.outputs = get_city_trajectories(city=city, split=split, normalized=False)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):

        data = (self.inputs[idx], self.outputs[idx])
            
        if self.transform:
            data = torch.nn.functional.normalize(data)

        return data

# intialize a dataset
city = 'all' 
train_dataset  = ArgoverseDataset(city = city, split = "train")
val_dataset = ArgoverseDataset(city = city, split = "val")
test_dataset= get_city_trajectories(city = city, split = "test")[0]

## Create model

In [79]:
batch_size = 1  # batch size 
train_loader = DataLoader(train_dataset,batch_size=batch_size)

In [96]:
# This code is inspired by the code from cnvrg.io/pytorch-lstm/
from torch import nn, optim

class Model(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, batch_size, output_size):
        super(Model, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.batch_size = batch_size
        
        self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, num_layers=num_layers, batch_first=True)
        self.linear = nn.Linear(98, output_size * 2)
    
    def forward(self, x, hidden_state, cell_state):        
        output, (hidden, cell) = self.lstm(x, (hidden_state, cell_state))
        output = output.reshape(-1, 49 * 2)
        output = self.linear(output)
        output = output.reshape(1, 60, 2)
        return output, (hidden, cell)
        
    def init_state(self):
        return torch.zeros(self.num_layers, self.batch_size, self.hidden_size)

In [97]:
learning_rate = 0.001
weight_decay = 0.00001
num_epochs = 1
hidden_size = 2

In [98]:
criterion = nn.MSELoss()
model = Model(2, hidden_size, 1, batch_size, 60)
opt = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

## Train and Validate Model

In [99]:
import time
import math
import random

for epoch in range(num_epochs):
    if (epoch == 0):
        startTime = time.time()
    
    total_loss = 0
    for i_batch, sample_batch in enumerate(train_loader):
        hidden_state = model.init_state()
        cell_state = model.init_state()
        inp, out = sample_batch
        if (inp.shape[0] == batch_size):
            modelInput = inp[:,:49,:]
            preds, (hidden_state, cell_state) = model(modelInput.reshape(batch_size, 49, 2).float(), hidden_state, cell_state)
            loss = criterion(preds, out.float())        
            opt.zero_grad()
            loss.backward()
            opt.step()
            total_loss += loss.item()
            if (i_batch % 1000 == 0):
                print('step {}  RMSE: {} MSE: {}'.format(i_batch, math.sqrt(total_loss / len(train_dataset)), total_loss / len(train_dataset)))
    if (epoch == 0):
        endTime = time.time()
        print(endTime - startTime)
    print('epoch {}  RMSE: {} MSE: {}'.format(epoch, math.sqrt(total_loss / len(train_dataset)), total_loss / len(train_dataset)))

step 0  RMSE: 1.6560723869561664 MSE: 2.7425757508386948
step 1000  RMSE: 114.30771874308073 MSE: 13066.254564247249
step 2000  RMSE: 159.03592262521926 MSE: 25292.42468525473
step 3000  RMSE: 194.92416205273832 MSE: 37995.42895196219
step 4000  RMSE: 224.97028771907188 MSE: 50611.630356401976
step 5000  RMSE: 249.05391772433842 MSE: 62027.853933841536
step 6000  RMSE: 270.2085339105224 MSE: 73012.65179807394
step 7000  RMSE: 288.8171574813556 MSE: 83415.35045561015
step 8000  RMSE: 306.12078967041305 MSE: 93709.93786843725
step 9000  RMSE: 321.0985534795301 MSE: 103104.28104664663
step 10000  RMSE: 335.6247183509674 MSE: 112643.95156816617
step 11000  RMSE: 350.037717264371 MSE: 122526.40350765173
step 12000  RMSE: 362.7867458066366 MSE: 131614.22293296916
step 13000  RMSE: 374.64197861917864 MSE: 140356.6121436931
step 14000  RMSE: 384.469773310998 MSE: 147817.0065898102
step 15000  RMSE: 394.73922753182404 MSE: 155819.05775242115
step 16000  RMSE: 403.50209706801166 MSE: 162813.9423

step 137000  RMSE: 2549.2577386998087 MSE: 6498715.018320863
step 138000  RMSE: 2552.193308731317 MSE: 6513690.685132906
step 139000  RMSE: 2555.2583549303454 MSE: 6529345.260441334
step 140000  RMSE: 2558.2663945237928 MSE: 6544726.945349765
step 141000  RMSE: 2560.958935039558 MSE: 6558510.666958947
step 142000  RMSE: 2563.7951258048433 MSE: 6573045.447100672
step 143000  RMSE: 2566.492068929612 MSE: 6586881.5398786
step 144000  RMSE: 2569.325568174251 MSE: 6601433.875273939
step 145000  RMSE: 2572.114259495393 MSE: 6615771.763899535
step 146000  RMSE: 2574.9353295719075 MSE: 6630291.951477587
step 147000  RMSE: 2577.6793088783975 MSE: 6644430.619419813
step 148000  RMSE: 2580.253890848548 MSE: 6657710.141239071
step 149000  RMSE: 2582.893298789657 MSE: 6671337.792932517
step 150000  RMSE: 2585.5156784878363 MSE: 6684891.323706416
step 151000  RMSE: 2587.993742717382 MSE: 6697711.612344323
step 152000  RMSE: 2590.6038437004195 MSE: 6711228.274995388
step 153000  RMSE: 2593.0073499021

In [102]:
val_loader = DataLoader(val_dataset,batch_size=batch_size)

val_loss = 0
for i_batch, sample_batch in enumerate(val_loader):
    hidden_state = model.init_state()
    cell_state = model.init_state()
    inp, out = sample_batch
    if (inp.shape[0] == batch_size):
        modelInput = inp[:,:49,:]
        preds, (hidden_state, cell_state) = model(modelInput.reshape(batch_size, 49, 2).float(), hidden_state, cell_state)
        loss = criterion(preds, out.float())        
        opt.zero_grad()
        loss.backward()
        opt.step()
        val_loss += loss.item()
        if (i_batch % 1000 == 0):
            print('step {}  RMSE: {} MSE: {}'.format(i_batch, math.sqrt(val_loss / len(train_dataset)), val_loss / len(train_dataset)))

print('loss: {}'.format(math.sqrt(val_loss / len(val_dataset))))

step 0  RMSE: 3.7489020743625714 MSE: 14.054266763159992
step 1000  RMSE: 113.99485911318169 MSE: 12994.827904234144
step 2000  RMSE: 159.76461369025654 MSE: 25524.731787596906
step 3000  RMSE: 192.74318064206795 MSE: 37149.933684020834
step 4000  RMSE: 222.6709489748429 MSE: 49582.35151735709
step 5000  RMSE: 250.32932792005337 MSE: 62664.77241690561
step 6000  RMSE: 274.80729065893763 MSE: 75519.04699930582
step 7000  RMSE: 298.252908849201 MSE: 88954.7976370098
step 8000  RMSE: 317.0495113154559 MSE: 100520.39262536941
step 9000  RMSE: 362.5456397183412 MSE: 131439.34087878125
step 10000  RMSE: 438.57939102939434 MSE: 192351.88223571438
step 11000  RMSE: 501.06247441340145 MSE: 251063.60326528057
step 12000  RMSE: 527.6317290949979 MSE: 278395.24154777726
step 13000  RMSE: 554.6987175807301 MSE: 307690.6672857066
step 14000  RMSE: 579.659447031494 MSE: 336005.07453285734
step 15000  RMSE: 604.4671011067082 MSE: 365380.47632034746
step 16000  RMSE: 626.2867395905854 MSE: 392235.08018

## Test Algorithm and Convert to CSV

In [115]:
outputs = np.zeros((test_dataset.shape[0], 120))
for inputIndex in range(0, test_dataset.shape[0]):
    hidden_state = model.init_state()
    cell_state = model.init_state()
    output, (hidden_state, cell_state) = model(torch.tensor(test_dataset[inputIndex][:49]).reshape(1, 49, 2).float(), hidden_state, cell_state)
    output = output.reshape(1,120)
    for outputIndex in range (0, 120):
        outputs[inputIndex][outputIndex] = output[0][outputIndex]

In [116]:
import pandas as pd
columns = []
for index in range(0, 120):
    columns.append("v" + str(index))
citynames = ["austin", "miami", "pittsburgh", "dearborn", "washington-dc", "palo-alto"]
testDataAmounts = [6325, 7971, 6361, 3671, 3829, 1686]
rows = []
for arrayIndex in range(0, 6):
    for itemIndex in range(0, testDataAmounts[arrayIndex]):
        rows.append(str(itemIndex) + "_" + citynames[arrayIndex])

df = pd.DataFrame(outputs, index=rows, columns=columns)
df.to_csv('submission.csv', index=True, header=True)